In [9]:
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

# Carpetas
DATOS_LIMPIOS_DIR = Path("datos_limpios")
RESULTADOS_PV_DIR = Path("resultados_pv")

LOCALIDADES = ["Calama", "Salvador", "Vallenar"]

# Carga los resultados generales de simulación
df_resultados = pd.read_csv(RESULTADOS_PV_DIR / "pv_simulation_results.csv")
df_sens_lcoe = pd.read_csv(RESULTADOS_PV_DIR / "sensibilidad_lcoe.csv")
df_sens_npv = pd.read_csv(RESULTADOS_PV_DIR / "sensibilidad_npv.csv")

# Inicializa la app Dash
app = Dash(__name__)
app.title = "Dashboard Solar - Limpieza y Simulación"

app.layout = html.Div([
    html.H1("Dashboard Solar: Limpieza y Simulación"),
    html.Label("Selecciona una localidad:"),
    dcc.Dropdown(
        id="dropdown-localidad",
        options=[{"label": loc, "value": loc} for loc in LOCALIDADES],
        value=LOCALIDADES[0]
    ),
    html.H2("Datos TMY limpios (primeras filas)"),
    dash_table.DataTable(id="tabla-tmy", page_size=10, style_table={'overflowX': 'auto'}),
    html.H2("Gráficos de radiación horaria"),
    dcc.Graph(id="grafico-tmy"),
    html.H2("Resultados de Simulación PV"),
    dash_table.DataTable(id="tabla-resultados", page_size=5, style_table={'overflowX': 'auto'}),
    html.H2("Análisis de Sensibilidad LCOE"),
    dcc.Graph(id="grafico-tornado-lcoe"),
    dash_table.DataTable(id="tabla-sens-lcoe", page_size=10, style_table={'overflowX': 'auto'}),
    html.H2("Análisis de Sensibilidad NPV"),
    dcc.Graph(id="grafico-tornado-npv"),
    dash_table.DataTable(id="tabla-sens-npv", page_size=10, style_table={'overflowX': 'auto'}),
])
def crear_tornado(df, parametro_impacto_bajo, parametro_impacto_alto, titulo):
    # Ordenar por el mayor impacto absoluto
    df = df.copy()
    df['max_impact'] = df[[parametro_impacto_bajo, parametro_impacto_alto]].abs().max(axis=1)
    df = df.sort_values('max_impact', ascending=True)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=df['Parameter'],
        x=df[parametro_impacto_bajo],
        orientation='h',
        name='Impacto bajo',
        marker_color='steelblue'
    ))
    fig.add_trace(go.Bar(
        y=df['Parameter'],
        x=df[parametro_impacto_alto],
        orientation='h',
        name='Impacto alto',
        marker_color='indianred'
    ))
    fig.update_layout(
        barmode='overlay',
        title=titulo,
        xaxis_title='Impacto (%)',
        yaxis_title='Parámetro',
        template='plotly_white',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig

@app.callback(
    Output("tabla-tmy", "data"),
    Output("tabla-tmy", "columns"),
    Output("grafico-tmy", "figure"),
    Output("tabla-resultados", "data"),
    Output("tabla-resultados", "columns"),
    Output("grafico-tornado-lcoe", "figure"),
    Output("tabla-sens-lcoe", "data"),
    Output("tabla-sens-lcoe", "columns"),
    Output("grafico-tornado-npv", "figure"),
    Output("tabla-sens-npv", "data"),
    Output("tabla-sens-npv", "columns"),
    Input("dropdown-localidad", "value")
)
def actualizar_dashboard(localidad):
    # Cargar TMY limpio
    tmy_file = DATOS_LIMPIOS_DIR / f"{localidad.lower()}_TMY_final.csv"
    df_tmy = pd.read_csv(tmy_file, skiprows=2)
    # Tabla TMY
    tmy_data = df_tmy.head(20).to_dict("records")
    tmy_columns = [{"name": i, "id": i} for i in df_tmy.columns]
    # Gráfico TMY
    fig_tmy = px.line(df_tmy, x=range(len(df_tmy)), y=["GHI", "DNI", "DHI"], labels={"value": "W/m²", "variable": "Componente"}, title=f"Radiación horaria - {localidad}")
    # Resultados PV
    df_res = df_resultados[df_resultados["Location"] == localidad]
    res_data = df_res.to_dict("records")
    res_columns = [{"name": i, "id": i} for i in df_res.columns]
    # Sensibilidad LCOE
    df_lcoe = df_sens_lcoe[df_sens_lcoe["Location"] == localidad]
    fig_lcoe = crear_tornado(df_lcoe, "Impact Low (%)", "Impact High (%)", f"Tornado LCOE - {localidad}")
    lcoe_data = df_lcoe.to_dict("records")
    lcoe_columns = [{"name": i, "id": i} for i in df_lcoe.columns]
    # Sensibilidad NPV
    df_npv = df_sens_npv[df_sens_npv["Location"] == localidad]
    fig_npv = crear_tornado(df_npv, "Impact Low (%)", "Impact High (%)", f"Tornado NPV - {localidad}")    
    npv_data = df_npv.to_dict("records")
    npv_columns = [{"name": i, "id": i} for i in df_npv.columns]
    return (tmy_data, tmy_columns, fig_tmy, res_data, res_columns, fig_lcoe, lcoe_data, lcoe_columns, fig_npv, npv_data, npv_columns)

# Ejecuta el dashboard en modo notebook
app.run(mode='inline')

In [7]:
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import pandas as pd
from pathlib import Path

# Carpetas
DATOS_LIMPIOS_DIR = Path("datos_limpios")
RESULTADOS_PV_DIR = Path("resultados_pv")

# Carga datos horarios y generales
df_hourly = pd.read_csv(RESULTADOS_PV_DIR / "pv_simulation_results_hourly.csv", parse_dates=["datetime"])
df_kpi = pd.read_csv(RESULTADOS_PV_DIR / "pv_simulation_results.csv")

# Si no tienes columna Year, extráela del datetime
if "Year" not in df_hourly.columns:
    df_hourly["Year"] = df_hourly["datetime"].dt.year

# Selector de país/localidad y año
localidades = df_hourly["Location"].unique()
anios = df_hourly["Year"].unique()

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard Solar Jupyter"),
    html.Div([
        html.Label("Selecciona país/localidad:"),
        dcc.Dropdown(
            id="dropdown-localidad",
            options=[{"label": loc, "value": loc} for loc in localidades],
            value=localidades[0]
        ),
        html.Label("Selecciona año:"),
        dcc.Dropdown(
            id="dropdown-anio",
            options=[{"label": str(a), "value": a} for a in anios],
            value=anios[0]
        ),
    ], style={"display": "flex", "gap": "2em"}),
    html.H2("Curva horaria de potencia (AC Power)"),
    dcc.Graph(id="grafico-potencia"),
    html.H2("KPIs diarios"),
    html.Div(id="kpi-diarios", style={"display": "flex", "gap": "3em"}),
])

@app.callback(
    Output("grafico-potencia", "figure"),
    Output("kpi-diarios", "children"),
    Input("dropdown-localidad", "value"),
    Input("dropdown-anio", "value")
)
def actualizar_dashboard(localidad, anio):
    # Filtra por localidad y año
    df_sel = df_hourly[(df_hourly["Location"] == localidad) & (df_hourly["Year"] == anio)].copy()
    # Curva horaria de potencia (puedes mostrar un día típico, o todo el año)
    fig = px.line(df_sel, x="datetime", y="AC Power (kW)", title=f"Potencia horaria - {localidad} {anio}")
    # KPIs diarios
    df_sel["date"] = df_sel["datetime"].dt.date
    energia_diaria = df_sel.groupby("date")["AC Power (kW)"].sum() * 1  # Si tus datos son horarios, esto da kWh/día
    energia_total = energia_diaria.sum()
    energia_prom = energia_diaria.mean()
    # LCOE diario (si tienes LCOE por año/ciudad, lo puedes mostrar aquí)
    lcoe = df_kpi[df_kpi["Location"] == localidad]["LCOE ($/kWh)"].values[0]
    # Factor de capacidad diario
    potencia_nominal = df_sel["AC Power (kW)"].max()  # O usa el valor real de tu planta
    cf_diario = energia_diaria / (potencia_nominal * 24)
    cf_prom = cf_diario.mean()
    # KPIs dinámicos
    kpis = [
        html.Div([
            html.H3("Energía diaria promedio"),
            html.P(f"{energia_prom:.2f} kWh/día")
        ]),
        html.Div([
            html.H3("LCOE"),
            html.P(f"{lcoe:.4f} $/kWh")
        ]),
        html.Div([
            html.H3("Factor de Capacidad promedio"),
            html.P(f"{cf_prom*100:.2f} %")
        ]),
    ]
    return fig, kpis

# Ejecuta el dashboard en Jupyter
app.run(mode='inline')

In [10]:
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

# Carpetas
DATOS_LIMPIOS_DIR = Path("datos_limpios")
RESULTADOS_PV_DIR = Path("resultados_pv")

# Carga datos horarios y generales
df_hourly = pd.read_csv(RESULTADOS_PV_DIR / "pv_simulation_results_hourly.csv", parse_dates=["datetime"])
df_kpi = pd.read_csv(RESULTADOS_PV_DIR / "pv_simulation_results.csv")
df_sens_lcoe = pd.read_csv(RESULTADOS_PV_DIR / "sensibilidad_lcoe.csv")
df_sens_npv = pd.read_csv(RESULTADOS_PV_DIR / "sensibilidad_npv.csv")

# Si no tienes columna Year, extráela del datetime
if "Year" not in df_hourly.columns:
    df_hourly["Year"] = df_hourly["datetime"].dt.year

localidades = df_hourly["Location"].unique()
anios = df_hourly["Year"].unique()

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard Solar Integrado"),
    html.Div([
        html.Label("Selecciona país/localidad:"),
        dcc.Dropdown(
            id="dropdown-localidad",
            options=[{"label": loc, "value": loc} for loc in localidades],
            value=localidades[0]
        ),
        html.Label("Selecciona año:"),
        dcc.Dropdown(
            id="dropdown-anio",
            options=[{"label": str(a), "value": a} for a in anios],
            value=anios[0]
        ),
    ], style={"display": "flex", "gap": "2em"}),
    html.H2("Curva horaria de potencia (AC Power)"),
    dcc.Graph(id="grafico-potencia"),
    html.H2("KPIs diarios"),
    html.Div(id="kpi-diarios", style={"display": "flex", "gap": "3em"}),
    html.H2("Datos TMY limpios (primeras filas)"),
    dash_table.DataTable(id="tabla-tmy", page_size=10, style_table={'overflowX': 'auto'}),
    html.H2("Gráficos de radiación horaria"),
    dcc.Graph(id="grafico-tmy"),
    html.H2("Resultados de Simulación PV"),
    dash_table.DataTable(id="tabla-resultados", page_size=5, style_table={'overflowX': 'auto'}),
    html.H2("Análisis de Sensibilidad LCOE"),
    dcc.Graph(id="grafico-tornado-lcoe"),
    dash_table.DataTable(id="tabla-sens-lcoe", page_size=10, style_table={'overflowX': 'auto'}),
    html.H2("Análisis de Sensibilidad NPV"),
    dcc.Graph(id="grafico-tornado-npv"),
    dash_table.DataTable(id="tabla-sens-npv", page_size=10, style_table={'overflowX': 'auto'}),
])
def crear_tornado(df, parametro_impacto_bajo, parametro_impacto_alto, titulo):
    # Ordenar por el mayor impacto absoluto
    df = df.copy()
    df['max_impact'] = df[[parametro_impacto_bajo, parametro_impacto_alto]].abs().max(axis=1)
    df = df.sort_values('max_impact', ascending=True)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=df['Parameter'],
        x=df[parametro_impacto_bajo],
        orientation='h',
        name='Impacto bajo',
        marker_color='steelblue'
    ))
    fig.add_trace(go.Bar(
        y=df['Parameter'],
        x=df[parametro_impacto_alto],
        orientation='h',
        name='Impacto alto',
        marker_color='indianred'
    ))
    fig.update_layout(
        barmode='overlay',
        title=titulo,
        xaxis_title='Impacto (%)',
        yaxis_title='Parámetro',
        template='plotly_white',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig
@app.callback(
    Output("grafico-potencia", "figure"),
    Output("kpi-diarios", "children"),
    Output("tabla-tmy", "data"),
    Output("tabla-tmy", "columns"),
    Output("grafico-tmy", "figure"),
    Output("tabla-resultados", "data"),
    Output("tabla-resultados", "columns"),
    Output("grafico-tornado-lcoe", "figure"),
    Output("tabla-sens-lcoe", "data"),
    Output("tabla-sens-lcoe", "columns"),
    Output("grafico-tornado-npv", "figure"),
    Output("tabla-sens-npv", "data"),
    Output("tabla-sens-npv", "columns"),
    Input("dropdown-localidad", "value"),
    Input("dropdown-anio", "value")
)
def actualizar_dashboard(localidad, anio):
    # --- Potencia horaria y KPIs ---
    df_sel = df_hourly[(df_hourly["Location"] == localidad) & (df_hourly["Year"] == anio)].copy()
    fig_pot = px.line(df_sel, x="datetime", y="AC Power (kW)", title=f"Potencia horaria - {localidad} {anio}")
    df_sel["date"] = df_sel["datetime"].dt.date
    energia_diaria = df_sel.groupby("date")["AC Power (kW)"].sum()
    energia_prom = energia_diaria.mean()
    energia_total = energia_diaria.sum()
    lcoe = df_kpi[df_kpi["Location"] == localidad]["LCOE ($/kWh)"].values[0]
    potencia_nominal = df_sel["AC Power (kW)"].max()
    cf_diario = energia_diaria / (potencia_nominal * 24)
    cf_prom = cf_diario.mean()
    kpis = [
        html.Div([
            html.H3("Energía diaria promedio"),
            html.P(f"{energia_prom:.2f} kWh/día")
        ]),
        html.Div([
            html.H3("LCOE"),
            html.P(f"{lcoe:.4f} $/kWh")
        ]),
        html.Div([
            html.H3("Factor de Capacidad promedio"),
            html.P(f"{cf_prom*100:.2f} %")
        ]),
    ]
    # --- TMY limpio y radiación ---
    tmy_file = DATOS_LIMPIOS_DIR / f"{localidad.lower()}_TMY_final.csv"
    df_tmy = pd.read_csv(tmy_file, skiprows=2)
    tmy_data = df_tmy.head(20).to_dict("records")
    tmy_columns = [{"name": i, "id": i} for i in df_tmy.columns]
    fig_tmy = px.line(df_tmy, x=range(len(df_tmy)), y=["GHI", "DNI", "DHI"], labels={"value": "W/m²", "variable": "Componente"}, title=f"Radiación horaria - {localidad}")
    # --- Resultados PV ---
    df_res = df_kpi[df_kpi["Location"] == localidad]
    res_data = df_res.to_dict("records")
    res_columns = [{"name": i, "id": i} for i in df_res.columns]
    # --- Sensibilidad LCOE ---
    df_lcoe = df_sens_lcoe[df_sens_lcoe["Location"] == localidad]
    fig_lcoe = crear_tornado(df_lcoe, "Impact Low (%)", "Impact High (%)", f"Tornado LCOE - {localidad}")
    lcoe_data = df_lcoe.to_dict("records")
    lcoe_columns = [{"name": i, "id": i} for i in df_lcoe.columns]
    # --- Sensibilidad NPV ---
    df_npv = df_sens_npv[df_sens_npv["Location"] == localidad]
    fig_npv = crear_tornado(df_npv, "Impact Low (%)", "Impact High (%)", f"Tornado NPV - {localidad}")    
    npv_data = df_npv.to_dict("records")
    npv_columns = [{"name": i, "id": i} for i in df_npv.columns]
    return (fig_pot, kpis, tmy_data, tmy_columns, fig_tmy, res_data, res_columns, fig_lcoe, lcoe_data, lcoe_columns, fig_npv, npv_data, npv_columns)

app.run(mode='inline')